In [1]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

output_notebook()

Loading BokehJS ...

# Tutorial \#3: Links with duration.

As our dataset in this tutorial, we will use a subset of the [MAWILab](http://www.fukuda-lab.org/mawilab/index.html) dataset.

MAWILab annotates traffic anomalies in the MAWI archive with four different labels

In [2]:
import pandas as pd
df = pd.read_csv('data.csv', names=['u', 'v', 'ts', 'tf'])
df.head()

,u,v,ts,tf
0,1157,1232,140,160
1,1157,1191,160,180
2,1157,1159,500,520
3,1157,1159,520,540
4,1159,1191,560,580


As we see our time-domain is that of floating point (*real*) numbers.

In [3]:
df.shape[0]

32424

## The TemporalLinkSet

This dataset finds its representative in our library, by what is notated as a **TemporalLinkSet**, which can be defined as a collection of tuples $(u, v, t_{s}, t_{f})$.  

Here we will initiliaze a temporal-link-set of links with duration, in continuous-time.  
We know that our dataset **doesn't** contain intervals that overlap for the same combination of $u, v$, so we want to signify this to our class-constructor by setting `disjoint_intervals=True`, or ommiting it as this is it's default value.
In order to signify that we are in continuous-time, we should also set the parameter `discrete` to `False`.

Note that suffix `DF` of our class name, indicates the fact that this implementation of a temporal-link-set is based on a dataframe.

In [4]:
from stream_graph import TemporalLinkSetDF
tls = TemporalLinkSetDF(df, discrete=True, disjoint_intervals=False)

In [5]:
tls.df.head()

,u,v,ts,tf
0,1159,1191,560,640
1,1157,1159,500,540
2,1144,1159,680,700
3,1105,1295,2260,2280
4,1157,1191,160,180


The first operation one can think of, is calculating the number of nodes that appear inside our temporal linkset.

In [6]:
tls.nodeset.size

75

Also we can calculate the total amount of time that temporal-links exists.

In [7]:
tls.size

662517

### Measures
So in order to see which node has the biggest traffic towards other nodes in the network, we can just calculate and sort the `out-degree`.

In [8]:
DF = pd.DataFrame(list(tls.degree_of(direction='out')), columns=['user', 'out-degree'])
DF = DF.set_index('user').sort_values(by=['out-degree'], ascending=False)
DF.head(5)

,out-degree
user,
1115,76573
1157,41482
1295,38941
1144,36763
1109,34319


Inversely, to see which node is the target of the biggest traffic, we can just calculate and sort the `in-degree`.

In [9]:
DF = pd.DataFrame(list(tls.degree_of(direction='in')), columns=['user', 'in-degree']).set_index('user')
DF.sort_values(by=['in-degree'], ascending=False).head(5)

,in-degree
user,
1210,60941
1295,36385
1207,32797
1221,31464
1191,27688


In order to see which connections between two nodes lasted the most, what you need is known as the duration of the link. To retrieve links as being uni-directional, we should set the parameter `direction` to `both`.

In [10]:
data = [(a, b, d) for ((a, b), d) in tls.duration_of(direction='both')]
DF = pd.DataFrame(data, columns=['src', 'dst', 'duration'])#.set_index(['src', 'dst'])

And plot the *max-5*.

In [11]:
DF.sort_values(by=['duration'], ascending=False).head(5)

,src,dst,duration
229,1115,1210,21467.0
670,1295,1625,11650.0
826,1295,1629,11531.0
185,1207,1210,11404.0
133,1115,1207,11247.0


So for a first figure let's calculate the number of links through time.

In [12]:
t, value = zip(*((t, v) for t, v in tls.m_at()))

Here `tls.m_at()` returns a sequence of changes. That is each time a new value `v` appears the time-stamp `t` where this value appears is provided. Time here is ascending and the information `f` signifies if the value is contained on exactly that time-stamp. That is because, the time-signature is continuous which translates to *bounds*.

In our case as the bounds are not provided, they are by default considered to be `both` closed. Their default value can be changed if the variable `default_closed` is set to one of `right`,  `left`, `neither`, `both`, upon initialization, or by providing an explicit column with the header name of `itype` with one of the above strings upon initialization, for each entry.

In [13]:
from bokeh.models.glyphs import Step
from bokeh.models import ColumnDataSource

p = figure(plot_width=400, plot_height=400)

# add a steps renderer
p.step(t, value, line_width=2, mode="before", color="green")

show(p)

But let's say we want to work on a subset of the linkstream. In order to do so we will use the `substream` method.  
Here we will extract the part of the final activity of the linkstream, signifying an interval in the important area from 0 to 100.

In [14]:
tls_sub = tls.substream(ts=[(int(1.4e+5), int(2.2e+5))])
tls_sub.df

,u,v,ts,tf
0,1365,1393,145080,145340
1,1365,1393,145500,145520
2,1365,1393,145540,145600
3,1365,1393,146080,146100
4,1365,1393,146980,147020
5,1365,1393,147360,147380
6,1365,1393,148780,148800
7,1365,1393,148820,148840
8,1365,1393,148900,148940
9,1207,1210,149760,149820


Here `s` and `f` are the column names corresponding to whether the *left* and *right* bound of the interval is open or closed.

Notice here, that unlike the **instantaneous** case, the `filter` function **will not** return something equivalent with the `substream`, as the second takes the intersection of existing links, that are bigger than the required duration, whereas filter keeps only the links that satisfy a property (throwing away all the rest).

In [15]:
tls_sub = tls.filter(lambda k: (k[2] >= 1.4e+5 and k[3] <= 2.2e+5))
tls_sub.df

,u,v,ts,tf
0,1365,1393,145080,145340
1,1365,1393,145500,145520
2,1365,1393,145540,145600
3,1365,1393,146080,146100
4,1365,1393,146980,147020
5,1365,1393,147360,147380
6,1365,1393,148780,148800
7,1365,1393,148820,148840
8,1207,1210,149760,149820
9,1207,1210,149840,149880


### Creating a StreamGraph
Now let's create a stream-graph!

A StreamGraph $S=(T, V, W, E)$ is a collection of four elements:
- $T $, a time-set
- $V $, a node-set
- $W\subseteq T \times V$, a temporal-node-stream
- $E\subseteq T \times V \otimes V$, a temporal-link-stream

To do so lets extract the nodeset and the timeset of our temporal-link-set:

In [16]:
ns, ts = tls.nodeset, tls.timeset

Now we would like to extract the minimal temporal-node-set, that is for each node that appears in our the node-set, we want it to dissapear whenever a link does not exist inside the temporal-link-set and anyone else.

In [17]:
tns = tls.minimal_temporal_nodeset

In [18]:
from stream_graph import StreamGraph
sg = StreamGraph(ns, ts, tns, tls)

The equivalent of what we did is equivalent with:

In [19]:
sg = tls.as_stream_graph_minimal

### Stream-Graph metrics
A stream-graph is usefull for what is known as calculating coverage.

Coverage is the percentage a measure satisfies its complete possible value.
For example in traditional graphs, a coverage measure is edge-density, which is an indicator of how much sparse a graph is.

The first measures we will calculate in this section is the total coverage of the temporal-linkset:
$$\frac{|E|}{\sum_{uv \in V\times V}|T_{u} \cap T_{v}|}$$

In [20]:
sg.temporal_linkset_coverage

0.19732252857640883

and the total coverage of the temporal-nodeset:
$$\frac{|W|}{|V\times T|}$$

In [21]:
sg.temporal_nodeset_coverage

0.07419512662717356

Now let's calculate and plot for each node $u$ it's time coverage"
$$\frac{|T_{u}|}{|T|}$$

In [22]:
from operator import itemgetter

us, tc = zip(*sorted(list(sg.time_coverage_node()), key=itemgetter(1), reverse=True))
x = list(range(1, len(us)+1))
p = figure()
p.vbar(x=x, top=tc, width=1)

show(p)

And we can do the same for links:
$$\frac{|T_{uv}|}{|T_{u} \cap T_{v}|}$$

In [23]:
link_coverage = sg.time_coverage_link()

In [24]:
from operator import itemgetter

links, tc = zip(*sorted(list(link_coverage), key=itemgetter(1), reverse=True))
x = list(range(1, len(links)+1))
p = figure()
p.step(x, tc, line_width=2, mode="before", color="#f46d43")

show(p)

Following the same logic, there are measures of coverage measured at a certain point in time.  
This measures are closely to the notion of density in static graphs.  
Thus we have node-coverage at time $t$:
$$\frac{|V_{t}|}{|V|}$$

In [26]:
t, v = zip(*sg.node_coverage_at())

In [27]:
from operator import itemgetter

p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

and we have link-coverage at time $t$:
$$\frac{|E_{t}|}{|V|^{2}}$$

In [29]:
t, v = zip(*((t, v) for t, v in sg.link_coverage_at()))

In [30]:
p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

Now, for measures concerning the degree, there is `neighbor_coverage`:
$$\frac{|N(u)|}{\sum_{v\in V}{|T_{u}\cap T_{v}|}}$$

In [31]:
nc = sg.neighbor_coverage()

In [32]:
from operator import itemgetter

links, tc = zip(*sorted(list(nc), key=itemgetter(1), reverse=True))
x = list(range(1, len(links)+1))
p = figure()
p.step(x, tc, line_width=2, mode="before", color="#f46d43")

show(p)

And of-course we can have the neighbor-coverage of a node at a certain time-stamp:

$$\frac{|N_{t}(u)|}{|V|}$$

In [33]:
nc = sg.neighbor_coverage_at()

Finally we can measure the mean_degree of the stream-graph over-time:
$$\frac{|E_{t}|}{|W_{t}|}$$

In [34]:
t, v = zip(*sg.mean_degree_at())

In [35]:
p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

## Cliques
A clique in a classic graph is defined as a set of nodes, where there exists a link between all pairs of nodes (subgraphs of density 1).

![image info](./graph-clique.png)

In a link-stream of continuous duration, a clique is defined as subset of the timeset and a subset of the nodes, where links exist between everybody.

![image info](./link-stream-clique.png)

In [36]:
# Delta cliques inside a twenty minute period
mxc = tls.get_maximal_cliques(direction='both')

And now let's sort and print the ten tiggest cliques.

In [37]:
# Efficient: whenever an actor exists it connects with all the other actors existing
sorted_cliques = sorted(list(mxc), key=lambda x:(len(x[0]), x[1][1] - x[1][0]), reverse=True)
sorted_cliques[:10]

[(frozenset({1098, 1109, 1115, 1207, 1658}), (176400, 176420)),
 (frozenset({1115, 1159, 1207, 1210, 1295}), (163700, 163720)),
 (frozenset({1098, 1109, 1115, 1207, 1210}), (176420, 176420)),
 (frozenset({1115, 1193, 1210, 1295, 1658}), (257360, 257360)),
 (frozenset({1115, 1144, 1164, 1207, 1210}), (77040, 77040)),
 (frozenset({1115, 1164, 1207, 1210, 1245}), (171020, 171020)),
 (frozenset({1098, 1115, 1149, 1207, 1210}), (89400, 89400)),
 (frozenset({1149, 1196, 1202, 1365}), (99600, 99680)),
 (frozenset({1144, 1157, 1193, 1245}), (168400, 168440)),
 (frozenset({1149, 1190, 1196, 1210}), (339780, 339820))]